# Semantic Relationship Extraction and Feature Engineering

By christian Spiteri Gauci and Adam Darmanin

## Literature and References

## Method

* Cleaning of data - Remove special characters, punctuation, and extra whitespace
* Tokenise using spacy
* Lemmatize tokens and remove stopwords
* extract relationships - using "nsubj", "ROOT", "dobj"
* Clean the extracted relationships - remove any entries that inlcude only special characters or numbers
* Remove entries that occur only once - and keep those that occur 3times or more in each subreddit 
* Remove entries that occur in more than one subreddit 


In [1]:
import pandas as pd
import numpy as np
import spacy
from collections import defaultdict, Counter
import re


import ast
from tqdm import tqdm
import nltk
from nltk.corpus import words as nltk_words


from lib.sanitze_util import clean_text_batch

tqdm.pandas()  # for progressbase in DFs.

nlp = spacy.load("en_core_web_sm")
nltk.download("words")


english_words = set(nltk_words.words())

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\chris\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


# Data Preprocessing

1. Clean stopwords
2. Lemmatize
3. Convert emoticons
4. Expand contractions

In [2]:
df = pd.read_csv("./data/mental_disorders_reddit.csv")

# Create a new DataFrame with specific columns
columns_needed = ["title", "selftext", "subreddit"]
df_subset = df[columns_needed].copy()

# Drop anything not a full peice of text in this dataset.
mask = df_subset.applymap(lambda x: isinstance(x, str) and x.strip() != "")
df_subset = df_subset[mask["title"] & mask["selftext"]]
df_subset["raw_title"] = df_subset["title"]
df_subset["raw_selftext"] = df_subset["selftext"]
df_subset["title"] = clean_text_batch(df_subset["title"].fillna("").tolist())
df_subset["selftext"] = clean_text_batch(df_subset["selftext"].fillna("").tolist())

C:\Users\chris\AppData\Local\Temp\ipykernel_3304\2031721653.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mask = df_subset.applymap(lambda x: isinstance(x, str) and x.strip() != "")
Cleaning Pipeline Token:   0%|          | 0/668031 [00:00<?, ?it/s]g:\My Drive\1. Educ\1. Masters\Year 1\ICS5111 - Mining Large Scale Data\Personal Assignment\gitrepo\mining-large-scale-data\lib\sanitze_util.py:88: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")
C:\Users\chris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\spacy\pipeline\lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+

# Feature Engineering

1. Semantic relations between words
2. Mental health labels in tokens

In [3]:
# borrowed from the research: https://github.com/zhukovanadezhda/psy-ner/tree/main
# Following paper: https://www.researchgate.net/publication/358779855_Deep_Learning-based_Detection_of_Psychiatric_Attributes_from_German_Mental_Health_Records
# see: https://spacy.io/usage/processing-pipelines
psy_ner = spacy.load("./model/psy_ner")
from nltk.sentiment import SentimentIntensityAnalyzer
from empath import Empath

lexicon = Empath()
sia = SentimentIntensityAnalyzer()

MH_NER = [
    "ANXIETY DISORDERS",
    "BIPOLAR DISORDERS",
    "DEPRESSIVE DISORDERS",
    "DISRUPTIVE IMPULSE-CONTROL, AND CONDUCT DISORDERS",
    "DISSOCIATIVE DISORDERS",
    "EATING DISORDERS",
    "NEURO-COGNITIVE DISORDERS",
    "NEURO-DEVELOPMENTAL DISORDERS",
    "OBSESSIVE-COMPULSIVE AND RELATED DISORDERS",
    "PERSONALITY DISORDERS",
    "PSYCHEDELIC DRUGS",
    "SCHIZOPHRENIA SPECTRUM AND OTHER PSYCHOTIC DISORDERS",
    "SEXUAL DYSFUNCTIONS",
    "SLEEP-WAKE DISORDERS",
    "SOMATIC SYMPTOM RELATED DISORDERS",
    "SUBSTANCE-RELATED DISORDERS",
    "SYMPTOMS",
    "TRAUMA AND STRESS RELATED DISORDERS",
]


def create_rel_feature(row):
    def _extract_relations(text):
        relations = []
        if not isinstance(text, str):
            return relations
        doc = nlp(text)

        for sent in doc.sents:
            for token in sent:
                if token.dep_ in ["nsubj", "dobj"]:
                    relations.append((token.head.text, token.dep_, token.text))

        return relations

    title_relations = _extract_relations(row["title"])
    text_relations = _extract_relations(row["selftext"])
    all_relations = title_relations + text_relations
    return all_relations


def create_psylabel_feature(row):
    def _extract_psy_labels(text):
        mh_labels = {}
        if not isinstance(text, str):
            return mh_labels

        doc = psy_ner(text)
        for ent in doc.ents:
            if ent.label_ in MH_NER:
                if ent.label_ not in mh_labels:
                    mh_labels[ent.label_] = set()
                mh_labels[ent.label_].add(ent.text)
        for label in mh_labels:
            mh_labels[label] = list(mh_labels[label])

        return mh_labels

    combined_text = row["title"] + " " + row["selftext"]
    combined_labels = _extract_psy_labels(combined_text)
    return combined_labels


# See: https://github.com/Ejhfast/empath-client/tree/master
EMPATH_CATS = [
    "help",
    "violence",
    "sleep",
    "medical_emergency",
    "cold",
    "hate",
    "cheerfulness",
    "aggression",
    "envy",
    "anticipation",
    "health",
    "pride",
    "nervousness",
    "weakness",
    "horror",
    "swearing_terms",
    "suffering",
    "sexual",
    "fear",
    "monster",
    "irritability",
    "exasperation",
    "ridicule",
    "neglect",
    "fight",
    "dominant_personality",
    "injury",
    "rage",
    "science",
    "work",
    "optimism",
    "warmth",
    "sadness",
    "emotional",
    "joy",
    "shame",
    "torment",
    "anger",
    "strength",
    "ugliness",
    "pain",
    "negative_emotion",
    "positive_emotion",
]


def create_sentiment_feature(row):
    def _get_vader_sentiment(text):
        score = sia.polarity_scores(text)
        return score["compound"] if score is not None else np.NaN

    combined_text = row["title"] + " " + row["selftext"]
    combined_labels = _get_vader_sentiment(combined_text)
    return combined_labels


def create_emotional_categories_scores_feature(row):
    def _get_empath_sentiment(text):
        scores = lexicon.analyze(text, categories=EMPATH_CATS, normalize=True)
        return {category: round(score, 2) for category, score in scores.items()}

    combined_text = row["title"] + " " + row["selftext"]
    combined_labels = _get_empath_sentiment(combined_text)
    return combined_labels


df_subset["psy_labels"] = df_subset.progress_apply(create_psylabel_feature, axis=1)
df_subset["sentiment"] = df_subset.progress_apply(create_sentiment_feature, axis=1)
df_subset["emotional_categories"] = df_subset.progress_apply(
    create_emotional_categories_scores_feature, axis=1
)
df_subset["semantic_relationships"] = df_subset.progress_apply(
    create_rel_feature, axis=1
)

# Cache it for further processing down the line.
df_subset.to_csv("./data/SemanticsRel.csv", index=False)
df_subset[
    [
        "title",
        "selftext",
        "psy_labels",
        "sentiment",
        "emotional_categories",
        "semantic_relationships",
    ]
].head(15)

LookupError: 
**********************************************************************
  Resource [93mvader_lexicon[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('vader_lexicon')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93msentiment/vader_lexicon.zip/vader_lexicon/vader_lexicon.txt[0m

  Searched in:
    - 'C:\\Users\\chris/nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.12_3.12.496.0_x64__qbz5n2kfra8p0\\nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.12_3.12.496.0_x64__qbz5n2kfra8p0\\share\\nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.12_3.12.496.0_x64__qbz5n2kfra8p0\\lib\\nltk_data'
    - 'C:\\Users\\chris\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**************************************************

# Create the Nodes

Nodes have edges, attributes for MH disorders, wieghts and sentiment.

In [5]:
clean_data = []

for idx, row in tqdm(df_subset.iterrows()):
    subreddit = row["subreddit"]
    labels = row["psy_labels"]
    relationships = row["semantic_relationships"]
    for rel in relationships:
        if len(rel) == 3:  # Ensure the tuple has three elements
            word1, dep, word2 = rel
            clean_data.append(
                {
                    "Subreddit": subreddit,
                    "Word1": word1,
                    "Dependency": dep,
                    "Word2": word2,
                    "MHlabels": labels,
                }
            )
        else:
            print(f"Issue with relationship: {rel}")

clean_df = pd.DataFrame(clean_data)
clean_df.head(15)

0it [00:00, ?it/s]


KeyError: 'psy_labels'

In [ ]:
# Grouping the DataFrame by 'Subreddit', 'Word1', 'Dependency', and 'Word2' and counting occurrences
grouped = (
    clean_df.groupby(["Subreddit", "Word1", "Dependency", "Word2"])
    .size()
    .reset_index(name="Occurrence")
)

# Filter out entries where Word1 and Word2 are not in the English dictionary
# filtered_relationships = multiple_subreddit_relationships[
#     (multiple_subreddit_relationships['Word1'].isin(english_words)) &
#     (multiple_subreddit_relationships['Word2'].isin(english_words))
# ]

# Define a regular expression pattern to match only alphanumeric words
pattern = re.compile(r"^[a-zA-Z]+$")

# Filter out entries where Word1 and Word2 contain only alphanumeric characters
filtered_relationships = grouped[
    (grouped["Word1"].str.match(pattern)) & (grouped["Word2"].str.match(pattern))
]

# Filtering relationships occurring more than three times within each subreddit
common_relationships = filtered_relationships[filtered_relationships["Occurrence"] > 3]


# Grouping the DataFrame by 'Word1', 'Dependency', and 'Word2' to count unique occurrences across subreddits
relationship_counts = (
    common_relationships.groupby(["Word1", "Dependency", "Word2"])
    .agg({"Subreddit": "nunique"})
    .reset_index()
)

# Filter relationships occurring in only one subreddit
single_subreddit_relationships = relationship_counts[
    relationship_counts["Subreddit"] == 1
]

# Merge to keep only the relationships occurring in one subreddit from common_relationships
filtered_common_relationships = pd.merge(
    common_relationships,
    single_subreddit_relationships,
    on=["Word1", "Dependency", "Word2"],
    how="inner",
)

print(filtered_common_relationships)
filtered_common_relationships.to_csv("SemanticsRelFiltered.csv", index=False)

    Subreddit_x      Word1 Dependency      Word2  Occurrence  Subreddit_y
0           BPD     advice       ROOT     advice          12            1
1           BPD    believe       ROOT    believe           5            1
2           BPD        bpd       ROOT        bpd          23            1
3           BPD      broke       ROOT      broke           4            1
4           BPD       came       ROOT       came           4            1
..          ...        ...        ...        ...         ...          ...
96          BPD       want      nsubj       love           4            1
97          BPD       want      nsubj     people           5            1
98          BPD       went       ROOT       went           9            1
99          BPD       wish       ROOT       wish           4            1
100         BPD  wondering       ROOT  wondering           4            1

[101 rows x 6 columns]


Some entries have word1 and word2 which are not actual words but numbers or mispelled words. Even though there was a chance that these were to be filtered due to occurrence should be more than 3, some weren't. At this point i was going to use the NLTK library to check for english words, but upon implementing, some important words like 'zyprexa' was eliminated since it's not included in the corpus. However this is an important entry and therefore should remain. a more crude filter, removing numbers only entries was implemented.

In [ ]:
english_vocab = set(w.lower() for w in nltk.corpus.words.words())
"zyprexa" in english_vocab

False